## Loading saved model and training on bioavailability

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from deepchem.feat import RDKitDescriptors
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
c:\Users\nina\anaconda3\envs\ai_chem\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\nina\anaconda3\envs\ai_chem\Lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loa

In [6]:
#load bioavailabity data
bio_avail_data = pd.read_csv("../Data/Bioavailibility.csv")
print(bio_avail_data.columns)

Index(['No', 'Name', 'Updated SMILES', '%F', 'logK(%F)', 'Category', 'Source',
       'MolW(Da)', 'NumHAcceptors', 'NumHDonors', 'LogP', 'Lipinski_rule'],
      dtype='object')


use code from chatgpt to make sure you have the same features.

In [18]:
# Function to generate features from SMILES strings using RDKit descriptors
def generate_features(smiles_list):
    featurizer = RDKitDescriptors()
    features = featurizer.featurize(smiles_list)
    return features

# Function to remove NaN features
def remove_nan_features(features):
    # Drop features containing invalid values
    valid_feature_indices = ~np.isnan(features).any(axis=0)
    features = features[:, valid_feature_indices]
    return features, valid_feature_indices

# Function to preprocess the data
def preprocess_data(data,y_name,smiles_name):
    # Remove rows with NaN solubility values
    data = data.dropna(subset=[y_name])

    # Get x and y data (x is the molecular descriptors, y is the solubility)
    y_data = data[y_name].values.astype(np.float32)

    # Generate features from SMILES data (get smiles from df)
    smiles = data[smiles_name]
    X_data = generate_features(smiles)

    return X_data, y_data

# Preprocess the first dataset
data1 = pd.read_csv("../Data/CuratedSol.csv")
X_data1, y_data1 = preprocess_data(data1,"Solubility","SMILES")

# Preprocess the second dataset
data2 = pd.read_csv("../Data/Bioavailibility.csv")
X_data2, y_data2 = preprocess_data(data2,"%F","Updated SMILES")


[14:53:11] WARNING: not removing hydrogen atom without neighbors
[14:53:11] WARNING: not removing hydrogen atom without neighbors
[14:53:11] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:13] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not removing hydrogen atom without neighbors
[14:53:15] WARNING: not r

In [19]:
# Get the indices of valid features in both datasets
valid_feature_indices = valid_feature_indices1 & valid_feature_indices2

# Filter features using the consistent set of indices
X_data1 = X_data1[:, valid_feature_indices]
X_data2 = X_data2[:, valid_feature_indices]


IndexError: boolean index did not match indexed array along dimension 1; dimension is 198 but corresponding boolean dimension is 210

In [ ]:
# Split data into training and validation using train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_data1, y_data1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_data2, y_data2, test_size=0.2, random_state=42)

# Scale x values
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

# Convert data to PyTorch tensors (like NumPy arrays but for PyTorch)
X_train_tensor1 = torch.tensor(X_train1, dtype=torch.float32)
y_train_tensor1 = torch.tensor(y_train1, dtype=torch.float32)
X_test_tensor1 = torch.tensor(X_test1, dtype=torch.float32)
y_test_tensor1 = torch.tensor(y_test1, dtype=torch.float32)

X_train_tensor2 = torch.tensor(X_train2, dtype=torch.float32)
y_train_tensor2 = torch.tensor(y_train2, dtype=torch.float32)
X_test_tensor2 = torch.tensor(X_test2, dtype=torch.float32)
y_test_tensor2 = torch.tensor(y_test2, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor1 = y_train_tensor1.view(-1, 1)
y_test_tensor1 = y_test_tensor1.view(-1, 1)
y_train_tensor2 = y_train_tensor2.view(-1, 1)
y_test_tensor2 = y_test_tensor2.view(-1, 1)


In [8]:
print(X_data1.shape)
print(X_data2.shape)
print(len(y_data))
print(np.std(y_data))

995
33.939842104248825


In [10]:
# Define the neural network architecture with multiple hidden layers
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(SolubilityPredictor, self).__init__()
        layers = []
        previous_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.ReLU())
            previous_dim = hidden_dim
        layers.append(nn.Linear(previous_dim, output_dim))
        self.network = nn.Sequential(*layers)
        self.activation = nn.ReLU() #best according to grid search

    def forward(self, x):
        return self.network(x)

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = [512,512,512]  # (512,512,512) Best according to grid search
output_dim = 1
learning_rate = 0.001
num_epochs = 1000
batch_size = 32

In [13]:
print(input_dim)

210


In [11]:
#loading saved model
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('solubility_model.pth'))

# Optionally, modify the final layer if the output dimension is different


RuntimeError: Error(s) in loading state_dict for SolubilityPredictor:
	size mismatch for network.0.weight: copying a param with shape torch.Size([512, 198]) from checkpoint, the shape in current model is torch.Size([512, 210]).

In [12]:
state_dict = torch.load('solubility_model.pth')
print(state_dict['network.0.weight'].shape)

torch.Size([512, 198])


In [ ]:
# Fine-tune the model on the new dataset. You might want to use a lower learning rate for fine-tuning.

learning_rate_finetune = 0.0001  # Lower learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=learning_rate_finetune)

# Create DataLoader for the new dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model...
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluate on the new test set
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {loss.item()}')


#### Notes:

Adjust the learning rate: When fine-tuning, it is often beneficial to use a lower learning rate to avoid large updates to the pre-trained weights.

Regularization: You can also consider adding dropout layers or other regularization techniques to avoid overfitting, especially when fine-tuning on a smaller dataset.

Layer freezing: Depending on your problem, you might want to freeze some layers (e.g., the first few layers) and only fine-tune the later layers or the final layer.